<h3 color="red">Conclusion: pix2struct not suitable for OCR</h3>

In [5]:
from pathlib import Path

from dataset.datasets_schema import Datasets

datasets_path = Path("../dataset/datasets.yaml")
datasets: Datasets = Datasets.from_yaml(datasets_path)
moodle_dataset = datasets["moodle"]
entries = moodle_dataset.document_storage.entries
print(len(entries))

821


In [6]:
pdf = next(entry for entry in entries.values() if entry.path and entry.path.exists() and entry.path.suffix == ".pdf")

In [14]:
from PIL import Image
import pymupdf

doc = pymupdf.open(pdf.path)
images = []

for page in doc:
    page: pymupdf.Page
    pix = pymupdf.utils.get_pixmap(page)
    pil = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)
    images.append(pil)

In [54]:
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor

model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-screen2words-base")
processor = Pix2StructProcessor.from_pretrained("google/pix2struct-screen2words-base")

In [55]:
inputs = processor(images=images[0], text="Text of page:\n", return_tensors="pt")

In [56]:
predictions = model.generate(**inputs)

In [57]:
print(processor.decode(predictions[0], skip_special_tokens=True))

page displaying text of a method
